# 01 - Thu thập dữ liệu (Crawling)

Mục tiêu: Thu thập dữ liệu sản phẩm/giá/đánh giá từ các sàn TMĐT tại Việt Nam (Shopee, Lazada, Tiki, ...).

**Lưu ý:** Không sử dụng dataset có sẵn. Dữ liệu phải được tự thu thập qua Web Scraping hoặc API.

## Kế hoạch thu thập
- Nguồn dữ liệu: Shopee/Lazada/Tiki
- Trường dữ liệu: tên sản phẩm, giá, số lượng bán, đánh giá, cửa hàng, thời gian
- Định dạng lưu: CSV/JSON

In [ ]:
# TODO: Cài đặt và kiểm tra thư viện
import requests
import pandas as pd